In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
pip install mxnet

     |████████████████████████████████| 25.4MB 35.9MB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [0]:
pip install gluon

     |████████████████████████████████| 133kB 26.0MB/s 
     |████████████████████████████████| 112kB 42.2MB/s 
     |████████████████████████████████| 4.8MB 42.1MB/s 
     |████████████████████████████████| 184kB 72.8MB/s 
     |████████████████████████████████| 92kB 16.2MB/s 
     |████████████████████████████████| 133kB 82.7MB/s 
     |████████████████████████████████| 81kB 14.5MB/s 
     |████████████████████████████████| 81kB 15.2MB/s 
     |████████████████████████████████| 102kB 17.3MB/s 
     |████████████████████████████████| 51kB 9.7MB/s 
     |████████████████████████████████| 61kB 10.9MB/s 
     |████████████████████████████████| 133kB 78.1MB/s 
     |████████████████████████████████| 122kB 71.9MB/s 
     |████████████████████████████████| 122kB 74.0MB/s 
     |████████████████████████████████| 317kB 42.8MB/s 
     |████████████████████████████████| 51kB 9.9MB/s 
     |████████████████████████████████| 112kB 76.9MB/s 
     |████████████████████████████████| 1.1MB 73.5MB/s 


In [0]:
pip install gluonnlp

     |████████████████████████████████| 245kB 47.6MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.8.1-cp36-none-any.whl size=293520 sha256=3f418167667efd42a17404e83b08ce3eeb2cbe06e9e3649dc1e82a8237e16632
  Stored in directory: /root/.cache/pip/wheels/3e/e7/3e/9cdf8ad7fce112fde2f4a52604045e5dd80f84d645bedb70c7
Successfully built gluonnlp


In [0]:
pip install plotly

In [0]:
import warnings
warnings.filterwarnings('ignore')

from mxnet import gluon
from mxnet import nd
import gluonnlp as nlp
import re
import numpy as np
import math
import plotly.graph_objects as go
import pickle

In [0]:
glove_6b100d = nlp.embedding.create('glove', source='glove.6B.100d')      #downloading glove embedding file

Embedding file glove.6B.100d.npz is not found. Downloading from Gluon Repository. This may take some time.


In [0]:
vocab = nlp.Vocab(nlp.data.Counter(glove_6b100d.idx_to_token))             #setting word embedding file
vocab.set_embedding(glove_6b100d)

In [0]:
#some basic functions

def cos_sim(x, y):
    return nd.dot(x, y) / (nd.norm(x) * nd.norm(y))


def dot_prod(x, y):
  return nd.dot(x, y)



def softmax(x):
    c = 0
    for i in x:
      c = c + np.exp(i)
    d = []
    for i in x:
      d.append(np.exp(i) / c)
    return d



def sig(x):
  return 1/(1+np.exp(-x))



def lg(x):
  return math.log(x)

In [0]:
def update_weights(h1, h2, u, w, b, mU, mW, mb):
  a, e, d = 0.01, 0.000001, 0.95
  mU, mW, mb = 0.13275610244547675, 0.3580114226800196, 0.01
  v_u, v_w, v_b = [], [], []
  cost1, cost2 = [], []
  retu, retw, retb = [], [], []
  loss1, loss2, loss3 = 0, 0, 0
  for i in range(len(h1)):
    cost1.append(h1[i]*lg(sig(h1[i])) + (1-h1[i])*lg(1-sig(h1[i])))
    cost2.append(h2[i]*lg(sig(h2[i])) + (1-h2[i])*lg(1-sig(h2[i])))
  c1 = np.array(cost1)
  c2 = np.array(cost2)
  for i in range(3):
    v_u.append(sum(np.gradient(c1, u[i])))
    v_w.append(sum(np.gradient(c1, w[i])))
    v_b.append(sum(np.gradient(c1, b[i])))
  for i in range(3,6):
    v_u.append(sum(np.gradient(c2, u[i])))
    v_w.append(sum(np.gradient(c2, w[i])))
    v_b.append(sum(np.gradient(c2, b[i])))
  dU, dW, db = sum(v_u)/6, sum(v_w)/6, sum(v_b)/6
  mU = d*mU + (1-d)*dU*dU
  mW = d*mW + (1-d)*dW*dW
  mb = d*mb + (1-d)*db*db
  loss1 = a*dU/(math.sqrt(mU)+e)
  loss2 = a*dW/(math.sqrt(mW)+e)
  loss3 = a*db/(math.sqrt(mb)+e)
  for i in range(0,6):
    retu.append(u[i] - loss1)
    retw.append(w[i] - loss2)
    retb.append(b[i] - loss3)
  return retu, retw, retb, mU, mW, mb

**ANGRY TRAINING**

In [0]:
U = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
W = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
b = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
mU = 0.3
mW = 0.3
mb = 0.01

In [0]:
ep1a,ep1b = [],[]
ep2a,ep2b = [],[]
ep3a,ep3b = [],[]
ep4a,ep4b = [],[]
ep5a,ep5b = [],[]
import pickle
indx = 0
s_t1_prev, s_t2_prev = 0, 0
h_co = 0
H1, H2 = [], []
for epoch in range(5):
  graph = 0
  with (open('/content/drive/My Drive/Colab Notebooks/Project_sem5/ANGER.pickle', "rb")) as openfile:       #opening tokenized file of specific label
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            #print(res)
            #print(len(res))
            prod = []
            alpha = []
            simil = []
            gru_in = []
            for i in res:
              if (dot_prod(vocab.embedding[i], vocab.embedding['angry']) == 0.0):
                continue
              prd = dot_prod(vocab.embedding[i], vocab.embedding['angry'])
              sil = cos_sim(vocab.embedding[i], vocab.embedding['angry'])
              prod.append(prd.asnumpy()[0])
              simil.append(sil.asnumpy()[0])
            #print(prod)
            #print(simil)
            alpha = softmax(prod)
              #print(alpha)
            gru_in = np.multiply(alpha, simil)
            #print(gru_in)
            for i in range(len(gru_in)):
              z_t1 = 1/(1+np.exp(-1*(U[0]*(gru_in[i]) + W[0]*(s_t1_prev) + b[0])))
              r_t1 = 1/(1+np.exp(-1*(U[1]*(gru_in[i]) + W[1]*(s_t1_prev) + b[1])))
              c_t1 = np.tanh(U[2]*(gru_in[i]) + W[2]*(s_t1_prev * r_t1) + b[2])
              s_t1 = (1 - z_t1) * c_t1 + z_t1 * s_t1_prev
              s_t1_prev = s_t1
            for i in range(len(gru_in)-1,-1,-1):
              z_t2 = 1/(1+np.exp(-1*(U[3]*(gru_in[i]) + W[3]*(s_t2_prev) + b[3])))
              r_t2 = 1/(1+np.exp(-1*(U[4]*(gru_in[i]) + W[4]*(s_t2_prev) + b[4])))
              c_t2 = np.tanh(U[5]*(gru_in[i]) + W[5]*(s_t2_prev * r_t2) + b[5])
              s_t2 = (1 - z_t2) * c_t2 + z_t2 * s_t2_prev
              s_t2_prev = s_t2
            #storing values for graph plotting
            if graph == 64:
              if epoch == 0:
                ep1a.append(s_t1)
                ep1b.append(s_t2)
                graph = 0
              elif epoch == 1:
                ep2a.append(s_t1)
                ep2b.append(s_t2)
                graph = 0
              elif epoch == 2:
                ep3a.append(s_t1)
                ep3b.append(s_t2)
                graph = 0
              elif epoch == 3:
                ep4a.append(s_t1)
                ep4b.append(s_t2)
                graph = 0
              else:
                ep5a.append(s_t1)
                ep5b.append(s_t2)
                graph = 0
            graph = graph + 1
            #storing values for weight updation
            if h_co == 64:
              U, W, b, mU, mW, mb = update_weights(H1, H2, U, W, b, mU, mW, mb)
              h_co = 0
              H1.clear(), H2.clear()
            else:
              H1.append(s_t1)
              H2.append(s_t2)
              h_co = h_co +1
            if epoch == 4:            #printing weights at 5th epoch
              indx = indx+1
              print("For ",indx+1," : ",z_t1,r_t1,c_t1,s_t1,sep='\t')
              print("For ",indx+1," : ",z_t2,r_t2,c_t2,s_t2,sep='\t',end='\n\n')
        except EOFError:
            break

For 	2	 : 	0.5779964597487074	0.5779964597487074	0.23089462352497192	0.29391022102084974
For 	2	 : 	0.5722416347889719	0.5722416347889719	0.21706533683601467	0.26302530796502976

For 	3	 : 	0.6903135824807431	0.6903135824807431	0.6433890818925522	0.3508341266312619
For 	3	 : 	0.5702764730521124	0.5702764730521124	0.21239200118589321	0.2525946869817557

For 	4	 : 	0.5759794489309277	0.5759794489309277	0.22602340753128686	0.28302757315291055
For 	4	 : 	0.5569315836062761	0.5569315836062761	0.18133741523171437	0.1833122143414263

For 	5	 : 	0.5692088572350086	0.5692088572350086	0.21286496654634846	0.24074808747237764
For 	5	 : 	0.5650829596219273	0.5650829596219273	0.2021364909640219	0.22134505544455768

For 	6	 : 	0.6855767354855037	0.6855767354855037	0.6340927392086508	0.3225969289797935
For 	6	 : 	0.5562298914478094	0.5562298914478094	0.17973801667196357	0.17974354757672936

For 	7	 : 	0.5804830857791708	0.5804830857791708	0.2369354505145365	0.30741276424703246
For 	7	 : 	0.56600509121

In [0]:
U_A, W_A, b_A = U, W, b
print(U_A, W_A, b_A, sep='\n')
print(len(ep5a))
eh1,eh2,eh3,eh4,eh5 = [],[],[],[],[]
for i in range(len(ep1a)):
  eh1.append(1)
  eh2.append(2)
  eh3.append(3)
  eh4.append(4)
  eh5.append(5)

import pickle
with open('/content/drive/My Drive/Colab Notebooks/Project_sem5/angry_weights.pickle', 'wb') as fp:         #storing weights of specific label
    pickle.dump(U, fp)
    pickle.dump(W, fp)
    pickle.dump(b, fp)
    pickle.dump(s_t1, fp)
    pickle.dump(s_t2, fp)
fp.close()


[0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471]
[0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471]
[0.14494800251411338, 0.14494800251411338, 0.14494800251411338, 0.14494800251411338, 0.14494800251411338, 0.14494800251411338]
32


In [0]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(x=eh1, y=ep1a, mode='markers', name='h1_ep1'))
fig1.add_trace(go.Scatter(x=eh1, y=ep1b, mode='markers', name='h2_ep1'))
fig1.add_trace(go.Scatter(x=eh2, y=ep2a, mode='markers', name='h1_ep2'))
fig1.add_trace(go.Scatter(x=eh2, y=ep2b, mode='markers', name='h2_ep2'))
fig1.add_trace(go.Scatter(x=eh3, y=ep3a, mode='markers', name='h1_ep3'))
fig1.add_trace(go.Scatter(x=eh3, y=ep3b, mode='markers', name='h2_ep3'))
fig1.add_trace(go.Scatter(x=eh4, y=ep4a, mode='markers', name='h1_ep4'))
fig1.add_trace(go.Scatter(x=eh4, y=ep4b, mode='markers', name='h4_ep4'))
fig1.add_trace(go.Scatter(x=eh5, y=ep5a, mode='markers', name='h1_ep5'))
fig1.add_trace(go.Scatter(x=eh5, y=ep5b, mode='markers', name='h2_ep5'))
fig1.show()

**HAPPY TRAINING**

In [0]:
U = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
W = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
b = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
mU = 0.3
mW = 0.3
mb = 0.01

In [0]:
ep1a,ep1b = [],[]
ep2a,ep2b = [],[]
ep3a,ep3b = [],[]
ep4a,ep4b = [],[]
ep5a,ep5b = [],[]
import pickle
indx = 0
s_t1_prev, s_t2_prev = 0, 0
h_co = 0
H1, H2 = [], []
for epoch in range(5):
  graph = 0
  with (open('/content/drive/My Drive/Colab Notebooks/Project_sem5/HAPPY.pickle', "rb")) as openfile:         #opening tokenized file of specific label
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            #print(res)
            #print(len(res))
            prod = []
            alpha = []
            simil = []
            gru_in = []
            for i in res:
              if (dot_prod(vocab.embedding[i], vocab.embedding['happy']) == 0.0):
                continue
              prd = dot_prod(vocab.embedding[i], vocab.embedding['happy'])
              sil = cos_sim(vocab.embedding[i], vocab.embedding['happy'])
              prod.append(prd.asnumpy()[0])
              simil.append(sil.asnumpy()[0])
            #print(prod)
            #print(simil)
            alpha = softmax(prod)
              #print(alpha)
            gru_in = np.multiply(alpha, simil)
            #print(gru_in)
            for i in range(len(gru_in)):
              z_t1 = 1/(1+np.exp(-1*(U[0]*(gru_in[i]) + W[0]*(s_t1_prev) + b[0])))
              r_t1 = 1/(1+np.exp(-1*(U[1]*(gru_in[i]) + W[1]*(s_t1_prev) + b[1])))
              c_t1 = np.tanh(U[2]*(gru_in[i]) + W[2]*(s_t1_prev * r_t1) + b[2])
              s_t1 = (1 - z_t1) * c_t1 + z_t1 * s_t1_prev
              s_t1_prev = s_t1
            for i in range(len(gru_in)-1,-1,-1):
              z_t2 = 1/(1+np.exp(-1*(U[3]*(gru_in[i]) + W[3]*(s_t2_prev) + b[3])))
              r_t2 = 1/(1+np.exp(-1*(U[4]*(gru_in[i]) + W[4]*(s_t2_prev) + b[4])))
              c_t2 = np.tanh(U[5]*(gru_in[i]) + W[5]*(s_t2_prev * r_t2) + b[5])
              s_t2 = (1 - z_t2) * c_t2 + z_t2 * s_t2_prev
              s_t2_prev = s_t2
            #storing weights for graph plotting
            if graph == 64:
              if epoch == 0:
                ep1a.append(s_t1)
                ep1b.append(s_t2)
                graph = 0
              elif epoch == 1:
                ep2a.append(s_t1)
                ep2b.append(s_t2)
                graph = 0
              elif epoch == 2:
                ep3a.append(s_t1)
                ep3b.append(s_t2)
                graph = 0
              elif epoch == 3:
                ep4a.append(s_t1)
                ep4b.append(s_t2)
                graph = 0
              else:
                ep5a.append(s_t1)
                ep5b.append(s_t2)
                graph = 0
            graph = graph + 1
            #storing weights for weight updation
            if h_co == 64:
              U, W, b, mU, mW, mb = update_weights(H1, H2, U, W, b, mU, mW, mb)
              h_co = 0
              H1.clear(), H2.clear()
            else:
              H1.append(s_t1)
              H2.append(s_t2)
              h_co = h_co +1
            if epoch == 4:        #printing weights in 5th epoch
              indx = indx+1
              print("For ",indx+1," : ",z_t1,r_t1,c_t1,s_t1,sep='\t')
              print("For ",indx+1," : ",z_t2,r_t2,c_t2,s_t2,sep='\t',end='\n\n')
        except EOFError:
            break

For 	2	 : 	0.5565744775937439	0.5565744775937439	0.1807754956404713	0.19105390826205487
For 	2	 : 	0.5533822642097469	0.5533822642097469	0.17353761445293903	0.1735567160304422

For 	3	 : 	0.5689432322704495	0.5689432322704495	0.20947844436221488	0.2604547372407616
For 	3	 : 	0.637005618383193	0.637005618383193	0.48589339428416617	0.28692626680650335

For 	4	 : 	0.5554065554875559	0.5554065554875559	0.1792526160365883	0.1822318604589118
For 	4	 : 	0.5667925988279316	0.5667925988279316	0.20441383315557032	0.2482029113124012

For 	5	 : 	0.5552951697276967	0.5552951697276967	0.17786640281879088	0.18402134456794728
For 	5	 : 	0.5535045360838105	0.5535045360838105	0.1738132608553739	0.17422439088825664

For 	6	 : 	0.5654931456679266	0.5654931456679266	0.2013690402841232	0.24083694820706286
For 	6	 : 	0.5560359088837461	0.5560359088837461	0.17954984685708705	0.1880891114958061

For 	7	 : 	0.5719034000886283	0.5719034000886283	0.21649938902006827	0.27744614027832326
For 	7	 : 	0.55935216328701

In [0]:
U_H, W_H, b_H = U, W, b
print(U_H, W_H, b_H, sep='\n')
print(len(ep5a))
eh1,eh2,eh3,eh4,eh5 = [],[],[],[],[]
for i in range(len(ep1a)):
  eh1.append(1)
  eh2.append(2)
  eh3.append(3)
  eh4.append(4)
  eh5.append(5)

with open('/content/drive/My Drive/Colab Notebooks/Project_sem5/happy_weights.pickle', 'wb') as fp:         #storing weights of specific label
    pickle.dump(U, fp)
    pickle.dump(W, fp)
    pickle.dump(b, fp)
    pickle.dump(s_t1, fp)
    pickle.dump(s_t2, fp)
fp.close()

[0.5008046020020307, 0.5008046020020307, 0.5008046020020307, 0.5008046020020307, 0.5008046020020307, 0.5008046020020307]
[0.5004783679213356, 0.5004783679213356, 0.5004783679213356, 0.5004783679213356, 0.5004783679213356, 0.5004783679213356]
[0.1397904097390755, 0.1397904097390755, 0.1397904097390755, 0.1397904097390755, 0.1397904097390755, 0.1397904097390755]
41


In [0]:
fig2 = go.Figure()

fig2.add_trace(go.Scatter(x=eh1, y=ep1a, mode='markers', name='h1_ep1'))
fig2.add_trace(go.Scatter(x=eh1, y=ep1b, mode='markers', name='h2_ep1'))
fig2.add_trace(go.Scatter(x=eh2, y=ep2a, mode='markers', name='h1_ep2'))
fig2.add_trace(go.Scatter(x=eh2, y=ep2b, mode='markers', name='h2_ep2'))
fig2.add_trace(go.Scatter(x=eh3, y=ep3a, mode='markers', name='h1_ep3'))
fig2.add_trace(go.Scatter(x=eh3, y=ep3b, mode='markers', name='h2_ep3'))
fig2.add_trace(go.Scatter(x=eh4, y=ep4a, mode='markers', name='h1_ep4'))
fig2.add_trace(go.Scatter(x=eh4, y=ep4b, mode='markers', name='h4_ep4'))
fig2.add_trace(go.Scatter(x=eh5, y=ep5a, mode='markers', name='h1_ep5'))
fig2.add_trace(go.Scatter(x=eh5, y=ep5b, mode='markers', name='h2_ep5'))
fig2.show()

**DEPRESSION TRAINING**

In [0]:
U = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
W = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
b = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
mU = 0.3
mW = 0.3
mb = 0.01

In [0]:
ep1a,ep1b = [],[]
ep2a,ep2b = [],[]
ep3a,ep3b = [],[]
ep4a,ep4b = [],[]
ep5a,ep5b = [],[]
import pickle
indx = 0
s_t1_prev, s_t2_prev = 0, 0
h_co = 0
H1, H2 = [], []
for epoch in range(5):
  graph = 0
  with (open('/content/drive/My Drive/Colab Notebooks/Project_sem5/DEPRESSION.pickle', "rb")) as openfile:          #opening tokenized file of specific label
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            #print(res)
            #print(len(res))
            prod = []
            alpha = []
            simil = []
            gru_in = []
            for i in res:
              if (dot_prod(vocab.embedding[i], vocab.embedding['depression']) == 0.0):
                continue
              prd = dot_prod(vocab.embedding[i], vocab.embedding['depression'])
              sil = cos_sim(vocab.embedding[i], vocab.embedding['depression'])
              prod.append(prd.asnumpy()[0])
              simil.append(sil.asnumpy()[0])
            #print(prod)
            #print(simil)
            alpha = softmax(prod)
              #print(alpha)
            gru_in = np.multiply(alpha, simil)
            #print(gru_in)
            for i in range(len(gru_in)):
              z_t1 = 1/(1+np.exp(-1*(U[0]*(gru_in[i]) + W[0]*(s_t1_prev) + b[0])))
              r_t1 = 1/(1+np.exp(-1*(U[1]*(gru_in[i]) + W[1]*(s_t1_prev) + b[1])))
              c_t1 = np.tanh(U[2]*(gru_in[i]) + W[2]*(s_t1_prev * r_t1) + b[2])
              s_t1 = (1 - z_t1) * c_t1 + z_t1 * s_t1_prev
              s_t1_prev = s_t1
            for i in range(len(gru_in)-1,-1,-1):
              z_t2 = 1/(1+np.exp(-1*(U[3]*(gru_in[i]) + W[3]*(s_t2_prev) + b[3])))
              r_t2 = 1/(1+np.exp(-1*(U[4]*(gru_in[i]) + W[4]*(s_t2_prev) + b[4])))
              c_t2 = np.tanh(U[5]*(gru_in[i]) + W[5]*(s_t2_prev * r_t2) + b[5])
              s_t2 = (1 - z_t2) * c_t2 + z_t2 * s_t2_prev
              s_t2_prev = s_t2
            #storing weights for graph plotting
            if graph == 64:
              if epoch == 0:
                ep1a.append(s_t1)
                ep1b.append(s_t2)
                graph = 0
              elif epoch == 1:
                ep2a.append(s_t1)
                ep2b.append(s_t2)
                graph = 0
              elif epoch == 2:
                ep3a.append(s_t1)
                ep3b.append(s_t2)
                graph = 0
              elif epoch == 3:
                ep4a.append(s_t1)
                ep4b.append(s_t2)
                graph = 0
              else:
                ep5a.append(s_t1)
                ep5b.append(s_t2)
                graph = 0
            graph = graph + 1
            #storing weights for weights updation
            if h_co == 64:
              U, W, b, mU, mW, mb = update_weights(H1, H2, U, W, b, mU, mW, mb)
              h_co = 0
              H1.clear(), H2.clear()
            else:
              H1.append(s_t1)
              H2.append(s_t2)
              h_co = h_co +1
            if epoch == 4:        #printing weights in 5th epoch
              indx = indx+1
              print("For ",indx+1," : ",z_t1,r_t1,c_t1,s_t1,sep='\t')
              print("For ",indx+1," : ",z_t2,r_t2,c_t2,s_t2,sep='\t',end='\n\n')
        except EOFError:
            break

For 	2	 : 	0.5772292025822605	0.5772292025822605	0.2580134417332443	0.24424732618307385
For 	2	 : 	0.5798488839564384	0.5798488839564384	0.2640925377630554	0.2597420541749811

For 	3	 : 	0.5693205557917654	0.5693205557917654	0.22722050827115822	0.2298093166348222
For 	3	 : 	0.5700221693909883	0.5700221693909883	0.22884158003094993	0.23391284906292964

For 	4	 : 	0.5685959055192021	0.5685959055192021	0.22555011262804786	0.22557886562214327
For 	4	 : 	0.5687232221914144	0.5687232221914144	0.22586624186814086	0.22626491745585234

For 	5	 : 	0.5748808643020037	0.5748808643020037	0.24015984516709604	0.26255635489037443
For 	5	 : 	0.5729627317896913	0.5729627317896913	0.23567012691027645	0.25120687572881933

For 	6	 : 	0.5689055327909556	0.5689055327909556	0.22626719096448564	0.22737597899081302
For 	6	 : 	0.5687533846920744	0.5687533846920744	0.2259128645888073	0.22649733435465236

For 	7	 : 	0.5709322249199669	0.5709322249199669	0.2319663893734312	0.23669945977275986
For 	7	 : 	0.573709811

In [0]:
U_D, W_D, b_D = U, W, b
print(U_D, W_D, b_D, sep='\n')
print(len(ep5a))
eh1,eh2,eh3,eh4,eh5 = [],[],[],[],[]
for i in range(len(ep1a)):
  eh1.append(1)
  eh2.append(2)
  eh3.append(3)
  eh4.append(4)
  eh5.append(5)

with open('/content/drive/My Drive/Colab Notebooks/Project_sem5/depression_weights.pickle', 'wb') as fp:        #storing weights of specific label
    pickle.dump(U_D, fp)
    pickle.dump(W_D, fp)
    pickle.dump(b_D, fp)
    pickle.dump(s_t1, fp)
    pickle.dump(s_t2, fp)
fp.close()

[0.46623864182830554, 0.46623864182830554, 0.46623864182830554, 0.46623864182830554, 0.46623864182830554, 0.46623864182830554]
[0.47961405033516247, 0.47961405033516247, 0.47961405033516247, 0.47961405033516247, 0.47961405033516247, 0.47961405033516247]
[0.17663249271441697, 0.17663249271441697, 0.17663249271441697, 0.17663249271441697, 0.17663249271441697, 0.17663249271441697]
41


In [0]:
fig3 = go.Figure()

fig3.add_trace(go.Scatter(x=eh1, y=ep1a, mode='markers', name='h1_ep1'))
fig3.add_trace(go.Scatter(x=eh1, y=ep1b, mode='markers', name='h2_ep1'))
fig3.add_trace(go.Scatter(x=eh2, y=ep2a, mode='markers', name='h1_ep2'))
fig3.add_trace(go.Scatter(x=eh2, y=ep2b, mode='markers', name='h2_ep2'))
fig3.add_trace(go.Scatter(x=eh3, y=ep3a, mode='markers', name='h1_ep3'))
fig3.add_trace(go.Scatter(x=eh3, y=ep3b, mode='markers', name='h2_ep3'))
fig3.add_trace(go.Scatter(x=eh4, y=ep4a, mode='markers', name='h1_ep4'))
fig3.add_trace(go.Scatter(x=eh4, y=ep4b, mode='markers', name='h4_ep4'))
fig3.add_trace(go.Scatter(x=eh5, y=ep5a, mode='markers', name='h1_ep5'))
fig3.add_trace(go.Scatter(x=eh5, y=ep5b, mode='markers', name='h2_ep5'))
fig3.show()

**FEAR TRAINING**

In [0]:
U = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
W = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
b = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
mU = 0.3
mW = 0.3
mb = 0.01

In [0]:
ep1a,ep1b = [],[]
ep2a,ep2b = [],[]
ep3a,ep3b = [],[]
ep4a,ep4b = [],[]
ep5a,ep5b = [],[]
import pickle
indx = 0
s_t1_prev, s_t2_prev = 0, 0
h_co = 0
H1, H2 = [], []
for epoch in range(5):
  graph = 0
  with (open('/content/drive/My Drive/Colab Notebooks/Project_sem5/FEAR.pickle', "rb")) as openfile:        #opening tokenized file of specific weights
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            #print(res)
            #print(len(res))
            prod = []
            alpha = []
            simil = []
            gru_in = []
            for i in res:
              if (dot_prod(vocab.embedding[i], vocab.embedding['fear']) == 0.0):
                continue
              prd = dot_prod(vocab.embedding[i], vocab.embedding['fear'])
              sil = cos_sim(vocab.embedding[i], vocab.embedding['fear'])
              prod.append(prd.asnumpy()[0])
              simil.append(sil.asnumpy()[0])
            #print(prod)
            #print(simil)
            alpha = softmax(prod)
              #print(alpha)
            gru_in = np.multiply(alpha, simil)
            #print(gru_in)
            for i in range(len(gru_in)):
              z_t1 = 1/(1+np.exp(-1*(U[0]*(gru_in[i]) + W[0]*(s_t1_prev) + b[0])))
              r_t1 = 1/(1+np.exp(-1*(U[1]*(gru_in[i]) + W[1]*(s_t1_prev) + b[1])))
              c_t1 = np.tanh(U[2]*(gru_in[i]) + W[2]*(s_t1_prev * r_t1) + b[2])
              s_t1 = (1 - z_t1) * c_t1 + z_t1 * s_t1_prev
              s_t1_prev = s_t1
            for i in range(len(gru_in)-1,-1,-1):
              z_t2 = 1/(1+np.exp(-1*(U[3]*(gru_in[i]) + W[3]*(s_t2_prev) + b[3])))
              r_t2 = 1/(1+np.exp(-1*(U[4]*(gru_in[i]) + W[4]*(s_t2_prev) + b[4])))
              c_t2 = np.tanh(U[5]*(gru_in[i]) + W[5]*(s_t2_prev * r_t2) + b[5])
              s_t2 = (1 - z_t2) * c_t2 + z_t2 * s_t2_prev
              s_t2_prev = s_t2
            #storing weights for graph plotting
            if graph == 64:
              if epoch == 0:
                ep1a.append(s_t1)
                ep1b.append(s_t2)
                graph = 0
              elif epoch == 1:
                ep2a.append(s_t1)
                ep2b.append(s_t2)
                graph = 0
              elif epoch == 2:
                ep3a.append(s_t1)
                ep3b.append(s_t2)
                graph = 0
              elif epoch == 3:
                ep4a.append(s_t1)
                ep4b.append(s_t2)
                graph = 0
              else:
                ep5a.append(s_t1)
                ep5b.append(s_t2)
                graph = 0
            graph = graph + 1
            #storing weights for weights updation
            if h_co == 64:
              U, W, b, mU, mW, mb = update_weights(H1, H2, U, W, b, mU, mW, mb)
              h_co = 0
              H1.clear(), H2.clear()
            else:
              H1.append(s_t1)
              H2.append(s_t2)
              h_co = h_co +1
            if epoch == 4:        #printing weights at 5th epoch
              indx = indx+1
              print("For ",indx+1," : ",z_t1,r_t1,c_t1,s_t1,sep='\t')
              print("For ",indx+1," : ",z_t2,r_t2,c_t2,s_t2,sep='\t',end='\n\n')
        except EOFError:
            break

For 	2	 : 	0.5698336774000203	0.5698336774000203	0.22023456053394078	0.23997183086824378
For 	2	 : 	0.5699840870346674	0.5699840870346674	0.2206064685480394	0.24075297236125576

For 	3	 : 	0.5686094711869484	0.5686094711869484	0.22286355893794219	0.22107826529794497
For 	3	 : 	0.5644650790286753	0.5644650790286753	0.20777480730265926	0.21072470230101709

For 	4	 : 	0.5652638961612002	0.5652638961612002	0.20979943150815566	0.2146506773201617
For 	4	 : 	0.5691898897434623	0.5691898897434623	0.2204084512224994	0.23270367531567943

For 	5	 : 	0.5635149114325297	0.5635149114325297	0.20559345875097815	0.2055882816065383
For 	5	 : 	0.5635139349971859	0.5635139349971859	0.2055861591067619	0.20559386226035137

For 	6	 : 	0.5636324410638971	0.5636324410638971	0.20585820391322726	0.20623301661895455
For 	6	 : 	0.6441443819866369	0.6441443819866369	0.5043952077385055	0.3119149401473268

For 	7	 : 	0.563726051980857	0.563726051980857	0.20609224413895275	0.2066970966612548
For 	7	 : 	0.5668000720363

In [0]:
U_F, W_F, b_F = U, W, b
print(U_F, W_F, b_F, sep='\n')
print(len(ep5a))
eh1,eh2,eh3,eh4,eh5 = [],[],[],[],[]
for i in range(len(ep1a)):
  eh1.append(1)
  eh2.append(2)
  eh3.append(3)
  eh4.append(4)
  eh5.append(5)

with open('/content/drive/My Drive/Colab Notebooks/Project_sem5/fear_weights.pickle', 'wb') as fp:        #storing weights of specific label
    pickle.dump(U, fp)
    pickle.dump(W, fp)
    pickle.dump(b, fp)
    pickle.dump(s_t1, fp)
    pickle.dump(s_t2, fp)
fp.close()

[0.5283030457065354, 0.5283030457065354, 0.5283030457065354, 0.5283030457065354, 0.5283030457065354, 0.5283030457065354]
[0.5283030457065354, 0.5283030457065354, 0.5283030457065354, 0.5283030457065354, 0.5283030457065354, 0.5283030457065354]
[0.16449197448340466, 0.16449197448340466, 0.16449197448340466, 0.16449197448340466, 0.16449197448340466, 0.16449197448340466]
47


In [0]:
fig4 = go.Figure()

fig4.add_trace(go.Scatter(x=eh1, y=ep1a, mode='markers', name='h1_ep1'))
fig4.add_trace(go.Scatter(x=eh1, y=ep1b, mode='markers', name='h2_ep1'))
fig4.add_trace(go.Scatter(x=eh2, y=ep2a, mode='markers', name='h1_ep2'))
fig4.add_trace(go.Scatter(x=eh2, y=ep2b, mode='markers', name='h2_ep2'))
fig4.add_trace(go.Scatter(x=eh3, y=ep3a, mode='markers', name='h1_ep3'))
fig4.add_trace(go.Scatter(x=eh3, y=ep3b, mode='markers', name='h2_ep3'))
fig4.add_trace(go.Scatter(x=eh4, y=ep4a, mode='markers', name='h1_ep4'))
fig4.add_trace(go.Scatter(x=eh4, y=ep4b, mode='markers', name='h4_ep4'))
fig4.add_trace(go.Scatter(x=eh5, y=ep5a, mode='markers', name='h1_ep5'))
fig4.add_trace(go.Scatter(x=eh5, y=ep5b, mode='markers', name='h2_ep5'))
fig4.show()

In [0]:
#####################################

In [0]:
#assighning H values (came after training) of each label

H1_A, H2_A = 0.24515719320893398, 0.2532973109535338
H1_H, H2_H = 0.21032951746028322, 0.19081880123924788
H1_D, H2_D = 0.2457876779911032, 0.2679380120205511
H1_F, H2_F = 0.2745960744505915, 0.2912388905168039

In [0]:
def h_value(h1, h2, text, word, U, W, b):         #function for H value calculation, used for finding max H value
  s_t1_prev, s_t2_prev = h1, h2
  s_t1, s_t2 = 0, 0
  prod = []
  alpha = []
  simil = []
  gru_in = []
  #H1, H2 = [], []
  for i in text:
    if (dot_prod(vocab.embedding[i], vocab.embedding[word]) == 0.0):
      continue
    prd = dot_prod(vocab.embedding[i], vocab.embedding[word])
    sil = cos_sim(vocab.embedding[i], vocab.embedding[word])
    prod.append(prd.asnumpy()[0])
    simil.append(sil.asnumpy()[0])
  #print(prod,'\n\n')
  #print(simil,'\n\n')
  alpha = softmax(prod)
  #print(alpha,'\n\n')
  gru_in = np.multiply(alpha, simil)
  #gru_in = np.squeeze(g)
  #print(gru_in)
  for i in range(len(gru_in)):
    z_t1 = 1/(1+np.exp(-1*(U[0]*(gru_in[i]) + W[0]*(s_t1_prev) + b[0])))
    r_t1 = 1/(1+np.exp(-1*(U[1]*(gru_in[i]) + W[1]*(s_t1_prev) + b[1])))
    c_t1 = np.tanh(U[2]*(gru_in[i]) + W[2]*(s_t1_prev * r_t1) + b[2])
    s_t1 = (1 - z_t1) * c_t1 + z_t1 * s_t1_prev
    s_t1_prev = s_t1
    #H1.append(s_t1)
  for i in range(len(gru_in)-1,-1,-1):
    z_t2 = 1/(1+np.exp(-1*(U[3]*(gru_in[i]) + W[3]*(s_t2_prev) + b[0])))
    r_t2 = 1/(1+np.exp(-1*(U[4]*(gru_in[i]) + W[4]*(s_t2_prev) + b[1])))
    c_t2 = np.tanh(U[5]*(gru_in[i]) + W[5]*(s_t2_prev * r_t1) + b[2])
    s_t2 = (1 - z_t1) * c_t2 + z_t2 * s_t2_prev
    s_t2_prev = s_t2
    #H2.append(s_t2)
  #print('\n',z_t1,r_t1,c_t1,s_t1,sep='     ')
  #print('\n',z_t2,r_t2,c_t2,s_t2,sep='     ')
  #print('\n\n')
  #print(H1,H2,sep='\n\n')
  return s_t1, s_t2

**ACCURACY**

In [0]:
#assigning GRU gate values

U_A = [0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471]
W_A = [0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471]
b_A = [0.14494800251411338, 0.14494800251411338, 0.14494800251411338, 0.14494800251411338, 0.14494800251411338, 0.14494800251411338]
U_H = [0.5008046020020307, 0.5008046020020307, 0.5008046020020307, 0.5008046020020307, 0.5008046020020307, 0.5008046020020307]
W_H = [0.5004783679213356, 0.5004783679213356, 0.5004783679213356, 0.5004783679213356, 0.5004783679213356, 0.5004783679213356]
b_H = [0.1397904097390755, 0.1397904097390755, 0.1397904097390755, 0.1397904097390755, 0.1397904097390755, 0.1397904097390755]
U_D = [0.46623864182830554, 0.46623864182830554, 0.46623864182830554, 0.46623864182830554, 0.46623864182830554, 0.46623864182830554]
W_D = [0.47961405033516247, 0.47961405033516247, 0.47961405033516247, 0.47961405033516247, 0.47961405033516247, 0.47961405033516247]
b_D = [0.17663249271441697, 0.17663249271441697, 0.17663249271441697, 0.17663249271441697, 0.17663249271441697, 0.17663249271441697]
U_F = [0.5283030457065354, 0.5283030457065354, 0.5283030457065354, 0.5283030457065354, 0.5283030457065354, 0.5283030457065354]
W_F = [0.5283030457065354, 0.5283030457065354, 0.5283030457065354, 0.5283030457065354, 0.5283030457065354, 0.5283030457065354]
b_F = [0.16449197448340466, 0.16449197448340466, 0.16449197448340466, 0.16449197448340466, 0.16449197448340466, 0.16449197448340466]

**HAPPY TRAINING ACCURACY**

In [0]:
h1_A, h2_A = [], []
h1_H, h2_H = [], []
h1_D, h2_D = [], []
h1_F, h2_F = [], []
count = 0
with (open('/content/drive/My Drive/Colab Notebooks/Project_sem5/HAPPY.pickle', "rb")) as openfile:     #opening training file for accuracy calculation
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            c1, c2 = h_value(H1_A, H2_A, res,'angry', U_A, W_A, b_A)
            c3, c4 = h_value(H1_H, H2_H, res,'happy', U_H, W_H, b_H)
            c5, c6 = h_value(H1_D, H2_D, res,'depression', U_D, W_D, b_D)
            c7, c8 = h_value(H1_F, H2_F, res,'fear', U_F, W_F, b_F)
            h1_A.append(c1), h2_A.append(c2)
            h1_H.append(c3), h2_H.append(c4)
            h1_D.append(c5), h2_D.append(c6)
            h1_F.append(c7), h2_F.append(c8)
            #print(c1,c2,'     ',c3,c4,'     ',c5,c6,sep='\t')
        except EOFError:
            break
#calculating sigmoid values of each h value
h1_A = [sig(x) for x in h1_A]
h2_A = [sig(x) for x in h2_A]
h1_H = [sig(x) for x in h1_H]
h2_H = [sig(x) for x in h2_H]
h1_D = [sig(x) for x in h1_D] 
h2_D = [sig(x) for x in h2_D]
h1_F = [sig(x) for x in h1_F]
h2_F = [sig(x) for x in h2_F]
print(len(h1_A))
for i in range(len(h1_A)):      #finding max h value for judging
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')
  if max(h1_A[i], h1_H[i], h1_D[i], h1_F[i]) == h1_H[i] or max(h2_A[i], h2_H[i], h2_D[i], h2_F[i]) == h2_H[i]:
    count = count + 1
print((count/2684)*100)
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')

2684
20.380029806259316


**HAPPY TEST ACCURACY**

In [0]:
h1_A, h2_A = [], []
h1_H, h2_H = [], []
h1_D, h2_D = [], []
h1_F, h2_F = [], []
count = 0
with (open('/content/drive/My Drive/Colab Notebooks/Project_sem5/happy_test.pickle', "rb")) as openfile:      #opening test file for accuracy calculation
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            c1, c2 = h_value(H1_A, H2_A, res,'angry', U_A, W_A, b_A)
            c3, c4 = h_value(H1_H, H2_H, res,'happy', U_H, W_H, b_H)
            c5, c6 = h_value(H1_D, H2_D, res,'depression', U_D, W_D, b_D)
            c7, c8 = h_value(H1_F, H2_F, res,'fear', U_F, W_F, b_F)
            h1_A.append(c1), h2_A.append(c2)
            h1_H.append(c3), h2_H.append(c4)
            h1_D.append(c5), h2_D.append(c6)
            h1_F.append(c7), h2_F.append(c8)
            #print(c1,c2,'     ',c3,c4,'     ',c5,c6,sep='\t')
        except EOFError:
            break
#calculating sigmoid values of each h value
h1_A = [sig(x) for x in h1_A]
h2_A = [sig(x) for x in h2_A]
h1_H = [sig(x) for x in h1_H]
h2_H = [sig(x) for x in h2_H]
h1_D = [sig(x) for x in h1_D] 
h2_D = [sig(x) for x in h2_D]
h1_F = [sig(x) for x in h1_F]
h2_F = [sig(x) for x in h2_F]
print(len(h1_A))
for i in range(len(h1_A)):        #finding max h value for judging
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')
  if max(h1_A[i], h1_H[i], h1_D[i], h1_F[i]) == h1_H[i] or max(h2_A[i], h2_H[i], h2_D[i], h2_F[i]) == h2_H[i]:
    count = count + 1
print((count/195)*100)
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')

195
22.564102564102566


**ANGRY TRAINING ACCURACY**

In [0]:
h1_A, h2_A = [], []
h1_H, h2_H = [], []
h1_D, h2_D = [], []
h1_F, h2_F = [], []
count = 0
with (open('/content/drive/My Drive/Colab Notebooks/Project_sem5/ANGER.pickle', "rb")) as openfile:         #opening training file for accuracy calculation
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            c1, c2 = h_value(H1_A, H2_A, res,'angry', U_A, W_A, b_A)
            c3, c4 = h_value(H1_H, H2_H, res,'happy', U_H, W_H, b_H)
            c5, c6 = h_value(H1_D, H2_D, res,'depression', U_D, W_D, b_D)
            c7, c8 = h_value(H1_F, H2_F, res,'fear', U_F, W_F, b_F)
            h1_A.append(c1), h2_A.append(c2)
            h1_H.append(c3), h2_H.append(c4)
            h1_D.append(c5), h2_D.append(c6)
            h1_F.append(c7), h2_F.append(c8)
            #print(c1,c2,'     ',c3,c4,'     ',c5,c6,sep='\t')
        except EOFError:
            break
#calculating sigmoid values of each h value
h1_A = [sig(x) for x in h1_A]
h2_A = [sig(x) for x in h2_A]
h1_H = [sig(x) for x in h1_H]
h2_H = [sig(x) for x in h2_H]
h1_D = [sig(x) for x in h1_D] 
h2_D = [sig(x) for x in h2_D]
h1_F = [sig(x) for x in h1_F]
h2_F = [sig(x) for x in h2_F]
print(len(h1_A))
for i in range(len(h1_A)):      #finding max h value for judging
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')
  if max(h1_A[i], h1_H[i], h1_D[i], h1_F[i]) == h1_A[i] or max(h2_A[i], h2_H[i], h2_D[i], h2_F[i]) == h2_A[i]:
    count = count + 1
print((count/2086)*100)
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')

2086
61.3614573346117


**ANGRY TEST ACCURACY**

In [0]:
h1_A, h2_A = [], []
h1_H, h2_H = [], []
h1_D, h2_D = [], []
h1_F, h2_F = [], []
count = 0
with (open('/content/drive/My Drive/Colab Notebooks/Project_sem5/angry_test.pickle', "rb")) as openfile:        #opening test file for accuracy calculation
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            c1, c2 = h_value(H1_A, H2_A, res,'angry', U_A, W_A, b_A)
            c3, c4 = h_value(H1_H, H2_H, res,'happy', U_H, W_H, b_H)
            c5, c6 = h_value(H1_D, H2_D, res,'depression', U_D, W_D, b_D)
            c7, c8 = h_value(H1_F, H2_F, res,'fear', U_F, W_F, b_F)
            h1_A.append(c1), h2_A.append(c2)
            h1_H.append(c3), h2_H.append(c4)
            h1_D.append(c5), h2_D.append(c6)
            h1_F.append(c7), h2_F.append(c8)
            #print(c1,c2,'     ',c3,c4,'     ',c5,c6,sep='\t')
        except EOFError:
            break
#calculating sigmoid values of each h value
h1_A = [sig(x) for x in h1_A]
h2_A = [sig(x) for x in h2_A]
h1_H = [sig(x) for x in h1_H]
h2_H = [sig(x) for x in h2_H]
h1_D = [sig(x) for x in h1_D] 
h2_D = [sig(x) for x in h2_D]
h1_F = [sig(x) for x in h1_F]
h2_F = [sig(x) for x in h2_F]
print(len(h1_A))
for i in range(len(h1_A)):      #finding max h value for judging
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')
  if max(h1_A[i], h1_H[i], h1_D[i], h1_F[i]) == h1_A[i] or max(h2_A[i], h2_H[i], h2_D[i], h2_F[i]) == h2_A[i]:
    count = count + 1
print((count/194)*100)
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')

194
53.608247422680414


**DEPRESSION TRAINING ACCURACY**

In [0]:
h1_A, h2_A = [], []
h1_H, h2_H = [], []
h1_D, h2_D = [], []
h1_F, h2_F = [], []
count = 0
with (open('/content/drive/My Drive/Colab Notebooks/Project_sem5/DEPRESSION.pickle', "rb")) as openfile:      #opening training file for acuracy calculation
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            c1, c2 = h_value(H1_A, H2_A, res,'angry', U_A, W_A, b_A)
            c3, c4 = h_value(H1_H, H2_H, res,'happy', U_H, W_H, b_H)
            c5, c6 = h_value(H1_D, H2_D, res,'depression', U_D, W_D, b_D)
            c7, c8 = h_value(H1_F, H2_F, res,'fear', U_F, W_F, b_F)
            h1_A.append(c1), h2_A.append(c2)
            h1_H.append(c3), h2_H.append(c4)
            h1_D.append(c5), h2_D.append(c6)
            h1_F.append(c7), h2_F.append(c8)
            #print(c1,c2,'     ',c3,c4,'     ',c5,c6,sep='\t')
        except EOFError:
            break
#calculating sigmoid values of each h value
h1_A = [sig(x) for x in h1_A]
h2_A = [sig(x) for x in h2_A]
h1_H = [sig(x) for x in h1_H]
h2_H = [sig(x) for x in h2_H]
h1_D = [sig(x) for x in h1_D] 
h2_D = [sig(x) for x in h2_D]
h1_F = [sig(x) for x in h1_F]
h2_F = [sig(x) for x in h2_F]
print(len(h1_A))
for i in range(len(h1_A)):        #finding max h value for judging
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')
  if max(h1_A[i], h1_H[i], h1_D[i], h1_F[i]) == h1_D[i] or max(h2_A[i], h2_H[i], h2_D[i], h2_F[i]) == h2_D[i]:
    count = count + 1
print((count/2122)*100)
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')

2122
66.49387370405277


**DEPRESSION TEST ACCURACY**

In [0]:
h1_A, h2_A = [], []
h1_H, h2_H = [], []
h1_D, h2_D = [], []
h1_F, h2_F = [], []
count = 0
with (open('/content/drive/My Drive/Colab Notebooks/Project_sem5/depression_test.pickle', "rb")) as openfile:       #opening test file for accuracy calculation
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            c1, c2 = h_value(H1_A, H2_A, res,'angry', U_A, W_A, b_A)
            c3, c4 = h_value(H1_H, H2_H, res,'happy', U_H, W_H, b_H)
            c5, c6 = h_value(H1_D, H2_D, res,'depression', U_D, W_D, b_D)
            c7, c8 = h_value(H1_F, H2_F, res,'fear', U_F, W_F, b_F)
            h1_A.append(c1), h2_A.append(c2)
            h1_H.append(c3), h2_H.append(c4)
            h1_D.append(c5), h2_D.append(c6)
            h1_F.append(c7), h2_F.append(c8)
            #print(c1,c2,'     ',c3,c4,'     ',c5,c6,sep='\t')
        except EOFError:
            break
#calculating sigmoid values of each h value
h1_A = [sig(x) for x in h1_A]
h2_A = [sig(x) for x in h2_A]
h1_H = [sig(x) for x in h1_H]
h2_H = [sig(x) for x in h2_H]
h1_D = [sig(x) for x in h1_D] 
h2_D = [sig(x) for x in h2_D]
h1_F = [sig(x) for x in h1_F]
h2_F = [sig(x) for x in h2_F]
print(len(h1_A))
for i in range(len(h1_A)):      #finding max h value for judging
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')
  if max(h1_A[i], h1_H[i], h1_D[i], h1_F[i]) == h1_D[i] or max(h2_A[i], h2_H[i], h2_D[i], h2_F[i]) == h2_D[i]:
    count = count + 1
print((count/199)*100)
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')

199
72.8643216080402


**FEAR TRAINING ACCURACY**

In [0]:
h1_A, h2_A = [], []
h1_H, h2_H = [], []
h1_D, h2_D = [], []
h1_F, h2_F = [], []
count = 0
with (open('/content/drive/My Drive/Colab Notebooks/Project_sem5/FEAR.pickle', "rb")) as openfile:        #opening training file for acuracy calculation
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            c1, c2 = h_value(H1_A, H2_A, res,'angry', U_A, W_A, b_A)
            c3, c4 = h_value(H1_H, H2_H, res,'happy', U_H, W_H, b_H)
            c5, c6 = h_value(H1_D, H2_D, res,'depression', U_D, W_D, b_D)
            c7, c8 = h_value(H1_F, H2_F, res,'fear', U_F, W_F, b_F)
            h1_A.append(c1), h2_A.append(c2)
            h1_H.append(c3), h2_H.append(c4)
            h1_D.append(c5), h2_D.append(c6)
            h1_F.append(c7), h2_F.append(c8)
            #print(c1,c2,'     ',c3,c4,'     ',c5,c6,sep='\t')
        except EOFError:
            break
#calculating sigmoid values of each h value
h1_A = [sig(x) for x in h1_A]
h2_A = [sig(x) for x in h2_A]
h1_H = [sig(x) for x in h1_H]
h2_H = [sig(x) for x in h2_H]
h1_D = [sig(x) for x in h1_D] 
h2_D = [sig(x) for x in h2_D]
h1_F = [sig(x) for x in h1_F]
h2_F = [sig(x) for x in h2_F]
print(len(h1_A))
for i in range(len(h1_A)):        #finding max h value for judging
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')
  if max(h1_A[i], h1_H[i], h1_D[i], h1_F[i]) == h1_F[i] and max(h2_A[i], h2_H[i], h2_D[i], h2_F[i]) == h2_F[i]:
    count = count + 1
print((count/3014)*100)
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')

3014
78.3344392833444


**FEAR TEST ACCURACY**

In [0]:
h1_A, h2_A = [], []
h1_H, h2_H = [], []
h1_D, h2_D = [], []
h1_F, h2_F = [], []
count = 0
with (open('/content/drive/My Drive/Colab Notebooks/Project_sem5/fear_test.pickle', "rb")) as openfile:     #opening test file for accuracy
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            c1, c2 = h_value(H1_A, H2_A, res,'angry', U_A, W_A, b_A)
            c3, c4 = h_value(H1_H, H2_H, res,'happy', U_H, W_H, b_H)
            c5, c6 = h_value(H1_D, H2_D, res,'depression', U_D, W_D, b_D)
            c7, c8 = h_value(H1_F, H2_F, res,'fear', U_F, W_F, b_F)
            h1_A.append(c1), h2_A.append(c2)
            h1_H.append(c3), h2_H.append(c4)
            h1_D.append(c5), h2_D.append(c6)
            h1_F.append(c7), h2_F.append(c8)
            #print(c1,c2,'     ',c3,c4,'     ',c5,c6,sep='\t')
        except EOFError:
            break
#calculating sigmoid values of each h value
h1_A = [sig(x) for x in h1_A]
h2_A = [sig(x) for x in h2_A]
h1_H = [sig(x) for x in h1_H]
h2_H = [sig(x) for x in h2_H]
h1_D = [sig(x) for x in h1_D] 
h2_D = [sig(x) for x in h2_D]
h1_F = [sig(x) for x in h1_F]
h2_F = [sig(x) for x in h2_F]
print(len(h1_A))
for i in range(len(h1_A)):        #finding max h value for judging
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')
  if max(h1_A[i], h1_H[i], h1_D[i], h1_F[i]) == h1_F[i] and max(h2_A[i], h2_H[i], h2_D[i], h2_F[i]) == h2_F[i]:
    count = count + 1
print((count/194)*100)
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')

194
81.95876288659794


In [0]:
#######################################################################################################################################